In [4]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import scipy.stats as stats
from statsmodels.stats.multitest import multipletests

In [5]:
def perform_regression_fdr(df_pheno, df_proteome, Protein_ID, platform_name, df_proteome_median, type):
    def run_regression(y, X, family):
        X = sm.add_constant(X)  # Add constant term for intercept
        if family == 'binomial':
            model = sm.GLM(y, X, family=sm.families.Binomial()).fit()
        else:
            model = sm.GLM(y, X, family=sm.families.Gaussian()).fit()
        return model

    df_proteome = df_proteome.to_numpy()

    # Define the dependent variables and their respective covariates
    regression_config = {
        'Age': (['Sex', 'Bmi'], 'gaussian'),
        'Sex': (['Age', 'Bmi'], 'binomial'),
        'Bmi': (['Age', 'Sex'], 'gaussian')
    }
    # Define common covariates
    common_covariates = ['FREG5_Ethnic_Group_I', 'FREG5_Ethnic_Group_M']
    results = {dep_var: [] for dep_var in regression_config}

    for i in range(df_proteome.shape[1]):
        protein_data = df_proteome[:, i]
        
        for dep_var, (covariates, family) in regression_config.items():
            X = np.column_stack((protein_data, df_pheno[covariates].values, df_pheno[common_covariates].values))
            model = run_regression(df_pheno[dep_var], X, family)
            results[dep_var].append([
                Protein_ID[i], model.params.iloc[1], model.bse.iloc[1], model.tvalues.iloc[1], model.pvalues.iloc[1]
            ])

    significance_value = 0.05
    bonferroni_threshold = significance_value / len(Protein_ID)
    significant_results = {}
    top_10_results = {}
    allresults = {}

    print(f"Number of samples: {df_pheno.shape[0]}")
    print(f"Number of proteins: {df_proteome.shape[1]}")

    for dep_var, result_list in results.items():
        columns = ['Protein_ID', 'Est', 'SE', 't_value', 'P']
        res_df = pd.DataFrame(result_list, columns=columns)

        # Print uncorrected p-value threshold
        print(f"Uncorrected p-value threshold: {significance_value}")
        print(f"Bonferroni correction threshold: {bonferroni_threshold}")

        res_df = res_df.dropna(subset=['P'])

        # FDR correction
        res_df['FDR_P'] = multipletests(res_df['P'], method='fdr_bh')[1]

        # Bonferroni correction
        res_df['Bonferroni_P'] = res_df['P'] < bonferroni_threshold

        # Find and print the FDR p-value threshold
        try:
            fdr_threshold = res_df[res_df['FDR_P'] < significance_value]['FDR_P'].max()
            print(f"FDR-corrected p-value threshold for {dep_var}: {fdr_threshold}")
        except:
            print(f"No significant results found for {dep_var} after FDR correction")

        print(f"Number of significant results for {dep_var} (uncorrected): {res_df[res_df['P'] < significance_value].shape[0]}")
        print(f"Number of significant results for {dep_var} (FDR corrected): {res_df[res_df['FDR_P'] < significance_value].shape[0]}")
        print(f"Number of significant results for {dep_var} (Bonferroni corrected): {res_df[res_df['Bonferroni_P']].shape[0]}")

        # Filter significant results using FDR-corrected p-values
        sig_res_df = res_df[res_df['FDR_P'] < significance_value]
        significant_results[dep_var] = sig_res_df

        # Annotate with Uniprot ID names
        uniprot = pd.read_csv("../data/uniprotkb_Human_AND_model_organism_9606_2024_05_20.tsv", sep='\t')
        uniprot = uniprot.iloc[:, [0, 3, 4, 7]]

        final_out = pd.merge(res_df, uniprot, left_on='Protein_ID', right_on='Entry', how='left')
        final_out = final_out.sort_values(by='FDR_P')
        final_out = pd.merge(final_out, df_proteome_median, left_on='Protein_ID', right_index=True, how='left')

        allresults[dep_var] = final_out
        # Save annotated results file
        output_file = f"./output/all/{platform_name}_{dep_var.lower()}_associations_{type}_fdr_corrected.csv"
        final_out.to_csv(output_file, sep='\t', index=False)
        print(f"Saved all {dep_var} associations to {output_file}")

        #save significant results
        sig_output_file = f"./output/significant/{platform_name}_{dep_var.lower()}_significant_associations_{type}_fdr_corrected.csv"
        sig_res_df.to_csv(sig_output_file, sep='\t', index=False)
        print(f"Saved significant {dep_var} associations to {sig_output_file}")

        # Store the top 10 significant results for each dependent variable
        top_10_results[dep_var] = final_out.head(10)

    # Lookup known associations by Uniprot ID (example: P15502)
    lookup_results = {dep_var: final_out[final_out['Protein_ID'] == "P15502"] for dep_var, final_out in allresults.items()}

    return significant_results, top_10_results, lookup_results, allresults


In [6]:
def process_olink_data(tech_rep, normalization):
    """
    Process Olink data.
    
    Parameters:
        tech_rep: int or str
            Technical replicate identifier to filter unique participants.
        normalization: bool
            Whether to apply inverse normal transformation to phenotypes.
            
    Returns:
        tuple: (df_pheno, df_proteome, df_proteome_median, df_unique)
    """
    # Read Olink data
    merged_df = pd.read_csv("../../data/olink/Preprocessed_data_intensity_normalized/Olink_Merged_All.csv")
    
    # Load phenotype data
    pheno = pd.read_csv("../data/HELIOS_Core_v4.csv")
    
    # Filter unique participants based on tech_rep
    df_unique = merged_df[(merged_df['tech_rep_id'] == "N") & (merged_df['bio_rep_id'] == "N") | (merged_df['tech_rep'] == tech_rep)]
    
    # Subset and factorize phenotypes
    pheno = (pheno
             .assign(Bmi=lambda x: 100*100*x['DBI14_Weight'] / (x['DBI13_Height']**2))
             .assign(Age=lambda x: x['FREG8_Age'],
                     Sex=lambda x: np.where(x['FREG7_Gender'] == "F", 1, 0))
             .loc[:, ['FREG1_Barcode', 'Age', 'Sex', 'FREG5_Ethnic_Group', 'Bmi']]
             .query('FREG5_Ethnic_Group != "O"'))

    pheno['Sex'] = pheno['Sex'].astype('category')
    
    # Convert 'FREG5_Ethnic_Group' to categorical dummy variables (drop_first=True like in original)
    pheno = pd.get_dummies(pheno, columns=['FREG5_Ethnic_Group'], drop_first=True)
    
    # Merge phenotype and proteomics data
    df_unique = pd.merge(pheno, df_unique, on='FREG1_Barcode')
    
    # Separate phenotype and proteomic datasets
    # Use the same column indexing as your original working code
    df_pheno = df_unique.iloc[:, :24]  # First 24 columns for phenotype
    df_proteome = df_unique.iloc[:, 24:]  # Remaining columns for proteome
    
    # Remove proteins with only one unique value (like in your original code)
    df_proteome = df_proteome.loc[:, df_proteome.apply(lambda x: x.nunique()) != 1]
    
    def inverse_normal_transform(series):
        ranks = stats.rankdata(series)  # Rank the data
        ranks = (ranks - 0.5) / len(series)  # Convert ranks to percentiles
        transformed = stats.norm.ppf(ranks)  # Apply inverse normal transformation
        return transformed

    # Do rank-based inverse normal transformation of the phenotypes Age and BMI to make them normally distributed
    if normalization == True:
        df_pheno['Age'] = inverse_normal_transform(df_pheno['Age'])
        df_pheno['Bmi'] = inverse_normal_transform(df_pheno['Bmi'])
    
    # Calculate median for proteomic data (same as your original code)
    df_proteome_median = df_proteome.median()
    df_proteome_median = df_proteome_median.reset_index()
    df_proteome_median.columns = ['Protein', 'Median']
    df_proteome_median.set_index('Protein', inplace=True)
    
    return df_pheno, df_proteome, df_proteome_median, df_unique

In [ ]:

# Process Olink data
df_pheno, df_proteome, df_proteome_median, df_unique = process_olink_data(1, True)
proteins= df_proteome.columns
display(df_pheno.head())
display(df_proteome.head())


In [10]:
#generate the column names from the common proteins file
common_proteins = pd.read_csv("common_proteins_soma_olink_thermo.txt", header=None)
common_proteins.columns = ['Proteins']
common_proteins_list = common_proteins['Proteins'].tolist()

#convert the common_proteins_list which are uniprot IDs to ALL matching olink IDs
def convert_uniprot_to_olink(uniprot_list):
    olink_to_uniprot = pd.read_csv("Olink_Assay_Annotation_All.csv")
    olink_to_uniprot = olink_to_uniprot.loc[:, ['OlinkID', 'UniProt']]
    
    # Filter rows where UniProt is in the common_proteins_list
    matching_proteins = olink_to_uniprot[olink_to_uniprot['UniProt'].isin(uniprot_list)]
    
    # Return all OlinkIDs that match
    return matching_proteins['OlinkID'].tolist()

common_proteins_olink = convert_uniprot_to_olink(common_proteins_list)

print(f"Number of common proteins (UniProt IDs): {len(common_proteins_list)}")
print(f"Number of corresponding Olink IDs: {len(common_proteins_olink)}")

# Optional: Check the mapping ratio
olink_annotation = pd.read_csv("Olink_Assay_Annotation_All.csv")
mapping_counts = olink_annotation[olink_annotation['UniProt'].isin(common_proteins_list)].groupby('UniProt').size()
print(f"Average Olink IDs per UniProt ID: {mapping_counts.mean():.2f}")
print(f"Max Olink IDs for a single UniProt ID: {mapping_counts.max()}")


Number of common proteins (UniProt IDs): 1740
Number of corresponding Olink IDs: 1742
Average Olink IDs per UniProt ID: 1.00
Max Olink IDs for a single UniProt ID: 3


In [11]:
def common(df_unique, use_common_samples=True, use_common_proteins=True):
    if use_common_samples:
        # Read common samples
        common_samples = pd.read_csv("common_samples.csv", header=None)
        common_samples.columns = ['FREG0_PID']

        # Filter the samples from df_unique
        df_unique_filter = df_unique[df_unique['FREG0_PID'].isin(common_samples['FREG0_PID'])]
    else:
        df_unique_filter = df_unique

    if use_common_proteins:
        # Filter the proteins from df_unique_filter using common_proteins
        df_unique_proteome = df_unique_filter.iloc[:, 23:]
        df_unique_proteome = df_unique_proteome[common_proteins_olink]
    else:
        df_unique_proteome = df_unique_filter.iloc[:, 23:]

    # Separate into phenotype and proteomic datasets
    df_pheno_filter = df_unique_filter.iloc[:, :23]
    df_proteome_filter = df_unique_proteome

    # Remove proteins with only one value
    df_proteome_filter = df_proteome_filter.loc[:, df_proteome_filter.apply(lambda x: x.nunique()) != 1]

    return df_pheno_filter, df_proteome_filter


In [12]:
def return_protein_list():
    #read xlsx file
    df1 = pd.read_excel('../../platform_analytes_list/explore-1536-assay-list-20210227-web-1.xlsx', header=1)
    uniprot_list_v1 = df1['Uniprot ID'].tolist()
   
    
    df2 = pd.read_excel('../../platform_analytes_list/olink-explore-3072-validation-data-results.xlsx', header=1)
    uniprot_list_v2 = df2['UniProt'].tolist()
    #display(df2.head())

    df3 = pd.read_csv('../../data/olink/Preprocessed_data_intensity_normalized/Olink_Assay_Annotation_All.csv',header=0)
    #display(df3.head())
    data3 = df3.loc[:, ['OlinkID', 'UniProt']]
    #display(data3.head())
    uniprot_list_v3 = df3['UniProt'].tolist()
    olinkid_v3 = df3['OlinkID'].tolist()


    uniprot_to_olinkid = dict(zip(data3['UniProt'], data3['OlinkID']))

    df_lod = pd.read_csv("../../data/olink/Preprocessed_data_intensity_normalized/Olink_HighProportionProteins_0.2_Intensity.csv")
    uniprot_lod = df_lod['UniProt'].to_list()
    olinkid_lod = df_lod['OlinkID'].to_list()

    #function to convert uniprot to olinkid
    def convert_uniprot_to_olinkid(uniprot_list, uniprot_to_olinkid):
        return list(filter(None, map(lambda uniprot: uniprot_to_olinkid.get(uniprot), uniprot_list)))
    
    olinkid_v1 = convert_uniprot_to_olinkid(uniprot_list_v1, uniprot_to_olinkid)
    olinkid_v2 = convert_uniprot_to_olinkid(uniprot_list_v2, uniprot_to_olinkid)
    uniprot_list_set1 = uniprot_list_v1
    uniprot_list_set2 = list(set(uniprot_list_v2) - set(uniprot_list_v1))
    uniprot_list_set3 = list(set(uniprot_list_v3) - set(uniprot_list_v2))
    olinkid_set1 = olinkid_v1
    olinkid_set2 = list(set(olinkid_v2) - set(olinkid_v1))
    olinkid_set3 = list(set(olinkid_v3) - set(olinkid_v2))


    return uniprot_list_set1, uniprot_list_set2, uniprot_list_set3, uniprot_lod, olinkid_set1, olinkid_set2, olinkid_set3, olinkid_lod

def return_set_breakdown (proteinlist, sequencelist):
    set1, set2, set3, lod, olinkid_set1, olinkid_set2, olinkid_set3, olinkid_lod = return_protein_list()
    set1_count = len(set(proteinlist) & set(set1))
    set2_count = len(set(proteinlist) & set(set2))
    set3_count = len(set(proteinlist) & set(set3))
    lod_count = len(set(proteinlist) & set(lod))

    olinkid_set1_count = len(set(sequencelist) & set(olinkid_set1))
    olinkid_set2_count = len(set(sequencelist) & set(olinkid_set2))
    olinkid_set3_count = len(set(sequencelist) & set(olinkid_set3))
    olinkkid_lod_count = len(set(sequencelist) & set(olinkid_lod))
    return set1_count, set2_count, set3_count, lod_count, olinkid_set1_count, olinkid_set2_count, olinkid_set3_count, olinkkid_lod_count

In [13]:

def olinkid_to_uniprot(olinklist):
    df = pd.read_csv("Olink_Assay_Annotation_All.csv")
    df = df.loc[:, ['OlinkID', 'UniProt']]
    #create a dictionary
    olink_to_uniprot = dict(zip(df['OlinkID'], df['UniProt']))
    return list(set(list(filter(None, map(lambda olink: olink_to_uniprot.get(olink), olinklist)))))

In [22]:
df_pheno_common, df_proteome_common = common(df_unique, use_common_samples=True, use_common_proteins=True)
Protein_ID_common = df_proteome_common.columns
Protein_ID_common_uniprot = olinkid_to_uniprot(Protein_ID_common)

# Perform regression analysis on common samples and proteins for Olink data
significant_results_common, top_10_common, lookup_result_common, results_common = perform_regression_fdr(
    df_pheno_common, 
    df_proteome_common, 
    Protein_ID_common, 
    "olink", 
    df_proteome_median, 
    type="common_protein_common_sample"
)

# Display results breakdown for Olink common data
for key in significant_results_common.keys():
    set1_count_total, set2_count_total, set3_count_total, lod_count_total, olinkid_set1_count_total, olinkid_set2_count_total, olinkid_set3_count_total, olinkid_lod_count_total = return_set_breakdown(Protein_ID_common_uniprot, Protein_ID_common)
    significant_olinkids = significant_results_common[key]['Protein_ID'].to_list()
    associated_proteins = olinkid_to_uniprot(significant_olinkids)
    
    print(f"\n{key} (Olink):")
    print("Number of total OlinkIDs: ", len(Protein_ID_common), "Number of total proteins: ", len(Protein_ID_common_uniprot))
    print("Number of lod OlinkIDs: ", olinkid_lod_count_total, "Number of lod proteins: ", lod_count_total)
    print(f"Number of significant OlinkIDs: {len(significant_olinkids)}", "Number of significant proteins: ", len(associated_proteins))
    set1_count_associated, set2_count_associated, set3_count_associated, lod_count_associated, olinkid_set1_count_associated, olinkid_set2_count_associated, olinkid_set3_count_associated, olinkid_lod_count_associated = return_set_breakdown(associated_proteins, significant_olinkids)
    
    print("Protein Breakdown")
    print(f"{key}: Set 1: {set1_count_associated}/{set1_count_total}, Set 2: {set2_count_associated}/{set2_count_total}, Set 3: {set3_count_associated}/{set3_count_total}")
    print(f"{key}: Below LoD: {lod_count_associated}/{lod_count_total}, Above LoD: {len(associated_proteins) - lod_count_associated}/{len(Protein_ID_common_uniprot) - lod_count_total}")
    
    print("OlinkID Breakdown")
    print(f"{key}: Set 1: {olinkid_set1_count_associated}/{olinkid_set1_count_total}, Set 2: {olinkid_set2_count_associated}/{olinkid_set2_count_total}, Set 3: {olinkid_set3_count_associated}/{olinkid_set3_count_total}")
    print(f"{key}: Below LoD: {olinkid_lod_count_associated}/{olinkid_lod_count_total}, Above LoD: {len(significant_olinkids) - olinkid_lod_count_associated}/{len(Protein_ID_common) - olinkid_lod_count_total}")

Number of samples: 46
Number of proteins: 1742
Uncorrected p-value threshold: 0.05
Bonferroni correction threshold: 2.870264064293915e-05
FDR-corrected p-value threshold for Age: 0.049367507167352685
Number of significant results for Age (uncorrected): 212
Number of significant results for Age (FDR corrected): 71
Number of significant results for Age (Bonferroni corrected): 21
Saved all Age associations to ./output/all/olink_age_associations_common_protein_common_sample_fdr_corrected.csv
Saved significant Age associations to ./output/significant/olink_age_significant_associations_common_protein_common_sample_fdr_corrected.csv
Uncorrected p-value threshold: 0.05
Bonferroni correction threshold: 2.870264064293915e-05
FDR-corrected p-value threshold for Sex: nan
Number of significant results for Sex (uncorrected): 168
Number of significant results for Sex (FDR corrected): 0
Number of significant results for Sex (Bonferroni corrected): 0
Saved all Sex associations to ./output/all/olink_sex

In [23]:
#write a function to return the number of unique uniprot IDs given a list of olink IDs
def count_unique_uniprot_ids_olink(olink_ids):
    olink_to_uniprot = pd.read_csv("Olink_Assay_Annotation_All.csv")
    olink_to_uniprot = olink_to_uniprot.loc[:, ['OlinkID', 'UniProt']]
    
    # Filter rows where OlinkID is in the olink_ids
    matching_proteins = olink_to_uniprot[olink_to_uniprot['OlinkID'].isin(olink_ids)]
    
    # Return the number of unique UniProt IDs
    return matching_proteins['UniProt'].nunique()

In [24]:
# return the number of unique uniprot IDs and total olink IDs in the significant results common
for key in significant_results_common.keys():
    unique_uniprot_count = count_unique_uniprot_ids_olink(significant_results_common[key]['Protein_ID'].tolist())
    total_olink_ids = len(significant_results_common[key]['Protein_ID'].tolist())
    print(f"{key}: Unique UniProt IDs: {unique_uniprot_count}, Total Olink IDs: {total_olink_ids}")

Age: Unique UniProt IDs: 71, Total Olink IDs: 71
Sex: Unique UniProt IDs: 0, Total Olink IDs: 0
Bmi: Unique UniProt IDs: 60, Total Olink IDs: 60


In [25]:
df_pheno_commonsample, df_proteome_commonsample = common(df_unique, use_common_samples=True, use_common_proteins=False)
Protein_ID_commonsample = df_proteome_commonsample.columns

def olinkid_to_uniprot(olinklist):
    df = pd.read_csv("../data/Olink_Assay_Annotation_All.csv")
    df = df.loc[:, ['OlinkID', 'UniProt']]
    #create a dictionary
    olink_to_uniprot = dict(zip(df['OlinkID'], df['UniProt']))
    return list(set(list(filter(None, map(lambda olink: olink_to_uniprot.get(olink), olinklist)))))

Protein_ID_commonsample_uniprot = olinkid_to_uniprot(Protein_ID_commonsample)

#length of the proteins
print(len(Protein_ID_commonsample_uniprot))

5401


In [30]:
df_pheno_commonsample, df_proteome_commonsample = common(df_unique, use_common_samples=True, use_common_proteins=False)
Protein_ID_commonsample = df_proteome_commonsample.columns
Protein_ID_commonsample_uniprot = olinkid_to_uniprot(Protein_ID_commonsample)
#lenth of proteins
print(f"Number of proteins: {len(Protein_ID_commonsample_uniprot)}")
print(f"Number of olinkids: {len(Protein_ID_commonsample)}")


significant_results_commonsample, top_10_commonsample, lookup_result_commonsample, results_commonsample = perform_regression_fdr(df_pheno_commonsample, df_proteome_commonsample, Protein_ID_commonsample, "olink", df_proteome_median, type="all_protein_common_sample")
for key in significant_results_commonsample.keys():
    set1_count_total, set2_count_total, set3_count_total, lod_count, apt_set1_count_total, apt_set2_count_total, apt_set3_count_total, apt_lod_count = return_set_breakdown(Protein_ID_commonsample_uniprot, Protein_ID_commonsample)
    significant_aptamers = significant_results_commonsample[key]['Protein_ID'].to_list()
    associated_proteins = olinkid_to_uniprot(significant_aptamers)
    print("Number of total aptamers: ", len(Protein_ID_commonsample), "Number of total proteins: ", len(Protein_ID_commonsample_uniprot))
    print("Number of lod aptamers: ", apt_lod_count, "Number of lod proteins: ", lod_count)
    print(f"Number of significant aptamers: {len(significant_aptamers)}", "Number of significant proteins: ", len(associated_proteins))
    set1_count_associated, set2_count_associated, set3_count_associated, lod_count_associated, apt_set1_count_associated, apt_set2_count_associated, apt_set3_count_associated, apt_lod_count_associated  = return_set_breakdown(associated_proteins, significant_aptamers)
    print("Protein Breakdown")
    print (f"{key}: Set 1: {set1_count_associated}/{set1_count_total}, Set 2: {set2_count_associated}/{set2_count_total}, Set 3: {set3_count_associated}/{set3_count_total}")
    print (f"{key}: Below LoD: {lod_count_associated}/{lod_count}, Above Lod: {len(associated_proteins) - (lod_count_associated)}/{len(Protein_ID_commonsample_uniprot) - lod_count}")
    print("Aptamer Breakdown")
    print (f"{key}: Set 1: {apt_set1_count_associated}/{apt_set1_count_total}, Set 2: {apt_set2_count_associated}/{apt_set2_count_total}, Set 3: {apt_set3_count_associated}/{apt_set3_count_total}")
    print (f"{key}: Below LoD: {apt_lod_count_associated}/{apt_lod_count}, Above Lod: {len(significant_aptamers) - (apt_lod_count_associated)}/{len(Protein_ID_commonsample) - apt_lod_count}")

Number of proteins: 5401
Number of olinkids: 5405
Number of samples: 46
Number of proteins: 5405
Uncorrected p-value threshold: 0.05
Bonferroni correction threshold: 9.250693802035154e-06
FDR-corrected p-value threshold for Age: 0.045973955609729375
Number of significant results for Age (uncorrected): 578
Number of significant results for Age (FDR corrected): 97
Number of significant results for Age (Bonferroni corrected): 26
Saved all Age associations to ./output/all/olink_age_associations_all_protein_common_sample_fdr_corrected.csv
Saved significant Age associations to ./output/significant/olink_age_significant_associations_all_protein_common_sample_fdr_corrected.csv
Uncorrected p-value threshold: 0.05
Bonferroni correction threshold: 9.250693802035154e-06
FDR-corrected p-value threshold for Sex: nan
Number of significant results for Sex (uncorrected): 452
Number of significant results for Sex (FDR corrected): 0
Number of significant results for Sex (Bonferroni corrected): 0
Saved al

In [31]:
# return the number of unique uniprot IDs and total olink IDs in the significant results common
for key in significant_results_commonsample.keys():
    unique_uniprot_count = count_unique_uniprot_ids_olink(significant_results_commonsample[key]['Protein_ID'].tolist())
    total_olink_ids = len(significant_results_commonsample[key]['Protein_ID'].tolist())
    print(f"{key}: Unique UniProt IDs: {unique_uniprot_count}, Total Olink IDs: {total_olink_ids}")

Age: Unique UniProt IDs: 97, Total Olink IDs: 97
Sex: Unique UniProt IDs: 0, Total Olink IDs: 0
Bmi: Unique UniProt IDs: 98, Total Olink IDs: 98
